# [RAG 문제]
- law_2.docx 파일을 읽고 Chroma 저장  
- LLM 질문 - > 답변
- 전세사기피해에 관한 법률 질문만 받도록.  
- 이외의 질문은 '답변을 할 수 없습니다.'라고 답변.  

In [ ]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from dotenv import load_dotenv

load_dotenv()


loader = Docx2txtLoader("law_2.docx")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, 
    chunk_overlap=150,
)

data_split = loader.load_and_split(text_splitter=text_splitter)

embedding = OpenAIEmbeddings(
    model="text-embedding-3-large",
)

database = Chroma(
    embedding_function=embedding,
    persist_directory='./chroma',                  
    collection_name='chroma-law',
)

    
query = input("질문을 입력하세요. 관련 없는 질문은 답변을 할 수 없습니다.")

retrieved_data = database.similarity_search(query=query)

prompt = '''
[identity]
- 당신은 전세사기피해 관련 법률 전문가입니다. 
- [context]을 참고해 사용자의 질문에 답변하세요. 
- [context]에 없는 내용에 대해서는 "관련 내용만 질문해주세요."라고 대답하세요. 

[context]
{retrieved_data}

[question]
{query}

'''

llm = ChatOpenAI(
    model_name='gpt-4.1-nano',
)

aimessage = llm.invoke(
    prompt.format(
        retrieved_data=retrieved_data,
        query=query
    )
)

aimessage

AIMessage(content='전세사기피해자에 대한 금융지원은 다음과 같이 이루어집니다.\n\n1. 긴급융자 및 지원: 국가와 지방자치단체는 전세사기피해자 및 임차인의 긴박한 주거안정을 위해 필요한 자금을 융자하거나 기타 지원을 제공할 수 있습니다. 특히, 임차인이 「주택임대차보호법」 제8조에 따른 우선변제를 받지 못하는 긴급 상황에서는 주택도시기금에서 임대차에 필요한 자금을 융자할 수 있습니다.\n\n2. 신용정보 관리 유예 및 삭제: 금융회사 등은 전세사기피해자 또는 임차인들의 보증금이 변제되지 않은 경우, 신용정보집중기관을 통해 관련 대출에 대한 채무불이행 또는 대위변제 등록을 유예하거나 삭제할 수 있는 조치를 취할 수 있습니다.\n\n3. 주택저당채권대상 지원: 「한국주택금융공사법」에 따른 혜택으로, 주택저당채권 대상에 준주택(예: 오피스텔)이 포함되어, 피해자들에게 더 나은 금융지원을 제공할 수 있도록 하고 있습니다.\n\n이와 같이 정부는 전세사기피해자의 조속한 주거안정을 위해 금융지원 조치를 강화하고 있으며, 피해자들이 신속하게 주거권을 회복할 수 있도록 다양한 금융지원 제도를 운영하고 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 319, 'prompt_tokens': 4193, 'total_tokens': 4512, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_38343a2

In [3]:
while True :
    query = input("질문을 입력하세요. 관련 없는 질문은 답변을 할 수 없습니다. S을 입력하면 대화 종료. ")
    if query.upper() == "S" :
        break

    aimessage = llm.invoke(
    prompt.format(
        retrieved_data=retrieved_data,
        query=query
        )
    )
    print(query)
    print(aimessage.content)
    print('*'*50)

 

전세사기피해자 금융지원에 대해 설명해주세요
전세사기피해자에 대한 금융지원은 주거안정과 피해 회복을 위해 국가 및 지방자치단체가 다양한 방식으로 지원하는 제도를 포함합니다. 구체적으로는 다음과 같습니다.

1. 자금 융자 및 지원: 국가와 지방자치단체는 전세사기 피해자와 긴급한 주거안정을 필요로 하는 임차인을 위해 필요한 자금을 융자하거나 기타 지원을 제공할 수 있습니다(제27조 1항). 특히, 「주택임대차보호법」에 따라 우선변제권이 행사되지 않아 긴급 지원이 필요한 피해자에게는 주택도시기금에서 임대차 자금을 융자하는 방안도 마련되어 있습니다(제27조 2항).

2. 신용정보 및 채무 관리: 금융회사들은 전세사기피해자 등의 보증금이 모두 변제되지 않은 경우 신용정보집중기관과 협력하여 해당 대출의 채무 불이행 및 대위변제 등록을 유예하거나 삭제하는 조치를 취할 수 있습니다(제27조 3항).

3. 긴급복지지원 연계: 전세사기피해자와 함께 생계·주거를 공유하는 가구원은 긴급복지지원법에 따른 긴급지원 대상자로 간주되어, 필요한 복지 지원을 받을 수 있습니다(제28조 1항).

4. 추가 지원책: 최근 법 개정을 통해, 공공임대주택 지원 및 비용 지원, 임대료 차액 지원, 임차주택 이주 지원 등 피해자를 위한 다양한 재정 지원과 주거 안정 방안이 마련되고 있으며, 이와 관련된 구체적인 실행 방안이 시행될 예정입니다(법률 개정 내용 참고).

이와 같은 금융지원은 피해자의 신속한 주거 안정과 경제적 부담 완화를 목적으로 하며, 구체적인 지원 대상, 방법, 조건 등은 별도 공고와 안내를 통해 진행됩니다. 자세한 내용은 해당 법률 또는 국토교통부 피해지원 관련 부서에 문의하시면 더욱 구체적인 지원 방안을 안내받을 수 있습니다.
**************************************************
오늘 날씨가 어떤가요?
관련 내용만 질문해주세요.
**************************************************
